In [1]:
import logging
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
import json
from minio import Minio
import unicodedata


In [2]:
minio_connection = ""

In [3]:
# Configuração básica de logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

In [4]:
# carregar para funcionar
try:
    minio_conn = json.loads(minio_connection)
except json.JSONDecodeError:
    with open("../variables/minio_connection.json", "r") as minio_connection_file:
        minio_conn = json.loads(minio_connection_file.read())

In [5]:
class LazySparkSession:
    packages = [
        "io.delta:delta-spark_2.13:4.0.0",
        "org.apache.hadoop:hadoop-aws:3.4.0",
        "com.amazonaws:aws-java-sdk-bundle:1.12.787",
    ]

    def __init__(self, access_key, secret_key, endpoint):
        self._access_key = access_key
        self._secret_key = secret_key
        self._endpoint = endpoint
        

    def start(
        self,
        app_name: str = "Airflow Spark Delta Minio App",
        executor_memory: str = "1g",
        driver_memory: str = "1g",
        driver_maxresultsize: str = "1g",
        master_url: str = "local[*]",
    ):

        builder = (
            SparkSession
            .Builder()
            .appName(app_name)
            .config("spark.hadoop.fs.s3a.access.key", self._access_key)
            .config("spark.hadoop.fs.s3a.secret.key", self._secret_key)
            .config("spark.hadoop.fs.s3a.endpoint", self._endpoint)
            .config("spark.hadoop.delta.enableFastS3AListFrom", "true")
            #
            .config("spark.executor.memory", executor_memory)
            .config("spark.driver.memory", driver_memory)
            .config("spark.driver.maxResultSize", driver_maxresultsize)
            #
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            #
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
            #
            .config("spark.jars.packages", ",".join(self.packages))
            .master(master_url)
        )

        return builder.getOrCreate()

In [6]:
spark = LazySparkSession(
    access_key=minio_conn.get("access_key"), 
    secret_key=minio_conn.get("key"), 
    endpoint=minio_conn.get("endpoint")
).start()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/20 17:09:57 WARN Utils: Your hostname, DESKTOP-EDEM2DH, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/08/20 17:09:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/edcarlos/projeto-lakehouse/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/edcarlos/.ivy2.5.2/cache
The jars for the packages stored in: /home/edcarlos/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d4d1d0b0-3201-426c-bc16-f505c6ee9403;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central

In [7]:
s3_client = None

try:
    endpoint_raw = minio_conn["endpoint"]
    access_key = minio_conn["access_key"]
    secret_key = minio_conn["key"]

    endpoint_sem_http = endpoint_raw.replace("http://", "").replace("https://", "")
    is_secure = endpoint_raw.startswith("https")

    s3_client = Minio(
        endpoint=endpoint_sem_http,
        access_key=access_key,
        secret_key=secret_key,
        secure=is_secure
    )

    logging.info("Cliente MinIO criado com sucesso.")

except KeyError as e:
    logging.error(f"Erro de configuração: chave ausente - {e}")
except Exception as e:
    logging.error(f"Erro ao inicializar o cliente MinIO: {e}")

2025-08-20 17:10:17,571 - INFO - Cliente MinIO criado com sucesso.


In [8]:
# Separa acento das letras e depois remove os acentos
def remover_acentos(txt: str) -> str:
    if txt is None:
        return None
    # Separa acento das letras
    nfkd = unicodedata.normalize('NFD', txt)
    # Remove os acentos (caracteres da categoria 'Mn')
    return "".join([c for c in nfkd if unicodedata.category(c) != 'Mn'])

In [9]:
def limpar_texto(txt):
    if txt is None:
        return None
    txt = unicodedata.normalize('NFD', txt)
    txt = ''.join([c for c in txt if unicodedata.category(c) != 'Mn'])
    txt = txt.strip().lower()
    txt = txt.replace('"', '')
    return txt

In [10]:
# Configurar os parâmetros para o arquivo que deseja processar
bucket = "landing"
pasta = "rfb/cnpj_qualificacoes"

input_path = f"s3a://{bucket}/{pasta}/*QUALSCSV"
output_path = f"s3a://bronze/{pasta}/"

logger.info(f"Lendo arquivo de entrada: {input_path}")

2025-08-20 17:10:17,597 - INFO - Lendo arquivo de entrada: s3a://landing/rfb/cnpj_qualificacoes/*QUALSCSV


In [11]:
try:
    # Delimitador correto
    df = spark.read.option("header", "false") \
                   .option("delimiter", ";") \
                   .option("encoding", "ISO-8859-1") \
                   .csv(input_path)
    logger.info("Arquivo lido com sucesso.")

    # Define nomes corretos das colunas
    colunas = [
        "codigo", 
        "descricao",
        ]
    
    df = df.toDF(*colunas)
    logger.info("Nomes de colunas atribuídos corretamente.")

except Exception as e:
    logger.error(f"Erro ao ler o arquivo {input_path}: {e}")
    raise

25/08/20 17:10:18 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
25/08/20 17:10:19 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://landing/rfb/cnpj_qualificacoes/*QUALSCSV.
java.io.FileNotFoundException: No such file or directory: s3a://landing/rfb/cnpj_qualificacoes/*QUALSCSV
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:4071)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3922)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.lambda$getFileStatus$26(S3AFileSystem.java:3899)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.invokeTrackingDuration(IOStatisticsBinding.ja

In [12]:
limpar_texto_udf = udf(limpar_texto, StringType())

In [13]:
# Aplicar a limpeza de texto em todas as colunas do tipo string
for c, tipo in df.dtypes:
    if tipo == "string":
        df = df.withColumn(c, limpar_texto_udf(col(c)))

logger.info("Textos das colunas padronizados: sem acentos, maiúsculos.")

2025-08-20 17:10:22,789 - INFO - Textos das colunas padronizados: sem acentos, maiúsculos.


In [14]:
# Salvar em Delta na camada bronze
df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(output_path)

logger.info(f"Arquivo QUALSCSV processado e salvo em Delta no caminho: {output_path}")

# Para verificar, conta linhas
count = df.count()
logger.info(f"Total de linhas: {count}")

25/08/20 17:10:25 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to rfb/cnpj_qualificacoes/part-00003-be0f049e-33cc-4834-9b70-9229fb0323fa-c000.snappy.parquet. This is Unsupported
25/08/20 17:10:27 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
2025-08-20 17:10:29,809 - INFO - Arquivo QUALSCSV processado e salvo em Delta no caminho: s3a://bronze/rfb/cnpj_qualificacoes/
2025-08-20 17:10:30,139 - INFO - Total de linhas: 1904


In [15]:
df.show(20, truncate=False)
spark.stop()

+------+----------------------------+
|codigo|descricao                   |
+------+----------------------------+
|00    |nao informada               |
|05    |administrador               |
|08    |conselheiro de administracao|
|09    |curador                     |
|10    |diretor                     |
|11    |interventor                 |
|12    |inventariante               |
|13    |liquidante                  |
|14    |mae                         |
|15    |pai                         |
|16    |presidente                  |
|17    |procurador                  |
|18    |secretario                  |
|19    |sindico (condominio)        |
|20    |sociedade consorciada       |
|21    |sociedade filiada           |
|22    |socio                       |
|23    |socio capitalista           |
|24    |socio comanditado           |
|25    |socio comanditario          |
+------+----------------------------+
only showing top 20 rows
